# Medicare Advantage Data Analysis 
## 1. Enrollment Data Analysis 
Shegofa Alizada
Assignment2/Submission1

1. Set the Working Directory

In [80]:
setwd("/home/salizad/econ470/a0/work/ma-data/ma/enrollment/Extracted Data/")

2. Load Necessary Libraries 

In [81]:
# Load necessary libraries
library(tidyr)
library(vroom)
library(dplyr)
library(ggplot2)
library(purrr)
#library(readr)  # For reading CSV files



3. Define the Years & a Month List for Analysis

In [82]:
# Define the year for analysis
years <- 2014:2019  # Example year, change as needed

# Create a month list based on the year
monthlist <- sprintf("%02d", 1:12) 

4. Define Functions for Data Reading
   

4.1 Reader for Contract Data

In [83]:
# Reader for contract data
read_contract <- function(path) {
  read_csv(
    path,
    skip = 1,
    col_names = c(
      "contractid", "planid", "org_type", "plan_type", "partd", "snp", "eghp",
      "org_name", "org_marketing_name", "plan_name", "parent_org", "contract_date"
    ),
    col_types = cols(
      contractid = col_character(),
      planid = col_double(),
      org_type = col_character(),
      plan_type = col_character(),
      partd = col_character(),
      snp = col_character(),
      eghp = col_character(),
      org_name = col_character(),
      org_marketing_name = col_character(),
      plan_name = col_character(),
      parent_org = col_character(),
      contract_date = col_character()
    ),
    show_col_types = FALSE,
    progress = FALSE
  )
}



4.2 Reader for Enrollment Data

In [84]:
# Reader for enrollment data
read_enroll <- function(path) {
  read_csv(
    path,
    skip = 1,
    col_names = c("contractid", "planid", "ssa", "fips", "state", "county", "enrollment"),
    col_types = cols(
      contractid = col_character(),
      planid = col_double(),
      ssa = col_double(),
      fips = col_double(),
      state = col_character(),
      county = col_character(),
      enrollment = col_double()
    ),
    na = "*",
    show_col_types = FALSE,
    progress = FALSE
  )
}

4.3 Load Monthly Data Function

In [85]:
load_month <- function(m, y) {
  c_path <- paste0("CPSC_Contract_Info_", y, "_", m, ".csv")
  e_path <- paste0("CPSC_Enrollment_Info_", y, "_", m, ".csv")

  # Print paths for debugging
  print(c_path)
  print(e_path)

  # Check if files exist
  if (!file.exists(c_path)) {
    warning(paste("Contract file does not exist:", c_path))
    return(data.frame())  # Return empty if contract file is not found
  }
  
  if (!file.exists(e_path)) {
    warning(paste("Enrollment file does not exist:", e_path))
    return(data.frame())  # Return empty if enrollment file is not found
  }

  # Use vroom to read the files and suppress column type messages
  contract.info <- vroom(c_path, show_col_types = FALSE) %>%
    distinct(`Contract ID`, `Plan ID`, .keep_all = TRUE)  # Use backticks for spaces

  enroll.info <- vroom(e_path, show_col_types = FALSE)  # Suppress column type messages

  if (nrow(enroll.info) == 0) {
    warning(paste("Enrollment file is empty:", e_path))
    return(data.frame())  # Return empty if enrollment is empty
  }

  if (nrow(contract.info) == 0) {
    warning(paste("Contract file is empty:", c_path))
    return(data.frame())  # Return empty if contract is empty
  }

  contract.info %>%
    left_join(enroll.info, by = c("Contract ID" = "Plan ID")) %>%  # Adjust join if names differ
    mutate(month = as.integer(m), year = y)
}
load_month("01", 2015)

[1] "CPSC_Contract_Info_2015_01.csv"
[1] "CPSC_Enrollment_Info_2015_01.csv"


Contract ID,Plan ID,Organization Type,Plan Type,Offers Part D,SNP Plan,EGHP,Organization Name,Organization Marketing Name,Plan Name,Parent Organization,Contract Effective Date,Contract Number,SSA State County Code,FIPS State County Code,State,County,Enrollment,month,year
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>
90091,NA,HCPP - 1833 Cost,HCPP - 1833 Cost,No,No,No,UNITED MINE WORKERS OF AMERICA HLTH & RETIREMENT,United Mine Workers of America Health & Retirement,NA,UMWA Health and Retirement Funds,02/01/1974 0:00:00,NA,NA,NA,NA,NA,NA,1,2015
E0654,801,Employer/Union Only Direct Contract PDP,Employer/Union Only Direct Contract PDP,Yes,No,Yes,IBT VOLUNTARY EMPLOYEE BENEFITS TRUST,TEAMStar Medicare Part D Prescription Drug Program,IBT Voluntary Employee Benefits Trust (Employer PDP),IBT Voluntary Employee Benefits Trust,01/01/2007 0:00:00,NA,NA,NA,NA,NA,NA,1,2015
E0654,802,Employer/Union Only Direct Contract PDP,Employer/Union Only Direct Contract PDP,Yes,No,Yes,IBT VOLUNTARY EMPLOYEE BENEFITS TRUST,TEAMStar Medicare Part D Prescription Drug Program,IBT Voluntary Employee Benefit Fund 802 (Employer PDP),IBT Voluntary Employee Benefits Trust,01/01/2007 0:00:00,NA,NA,NA,NA,NA,NA,1,2015
E0654,803,Employer/Union Only Direct Contract PDP,Employer/Union Only Direct Contract PDP,Yes,No,Yes,IBT VOLUNTARY EMPLOYEE BENEFITS TRUST,TEAMStar Medicare Part D Prescription Drug Program,IBT Voluntary Employee Benefit Fund 803 (Employer PDP),IBT Voluntary Employee Benefits Trust,01/01/2007 0:00:00,NA,NA,NA,NA,NA,NA,1,2015
E2630,801,Employer/Union Only Direct Contract PDP,Employer/Union Only Direct Contract PDP,Yes,No,Yes,C AND O EMPLOYEES' HOSPITAL ASSOCIATION,C and O Employees Hospital Association,C and O Employees Hospital Association (Employer PDP),C & O Employees' Hospital Association,01/01/2007 0:00:00,NA,NA,NA,NA,NA,NA,1,2015
E3014,801,Employer/Union Only Direct Contract PDP,Employer/Union Only Direct Contract PDP,Yes,No,Yes,PSERS HOP PROGRAM,Pennsylvania Public School Employees Retirement Sy,PSERS HOP (Employer PDP),Commonwealth of PA Pub Schools Retirement System,01/01/2007 0:00:00,NA,NA,NA,NA,NA,NA,1,2015
E4744,801,Employer/Union Only Direct Contract PDP,Employer/Union Only Direct Contract PDP,Yes,No,Yes,MODOT/MSHP MEDICAL AND LIFE INSURANCE PLAN,MISSOURI DEPARTMENT OF TRANSPORTATION,Missouri Department of Transportaion/ Highway Patr (Employer PDP),Missouri Highways and Transportation Commission,01/01/2007 0:00:00,NA,NA,NA,NA,NA,NA,1,2015
E7316,801,Employer/Union Only Direct Contract PDP,Employer/Union Only Direct Contract PDP,Yes,No,Yes,UNION PACIFIC RAILROAD EMPLOYES HEALTH SYSTEMS,Union Pacific Railroad Employes Health Systems,The UPREHS Prime Medicare Plan (Employer PDP),Union Pacific Railroad Employes Health Systems,01/01/2007 0:00:00,NA,NA,NA,NA,NA,NA,1,2015
E7848,801,Employer/Union Only Direct Contract PDP,Employer/Union Only Direct Contract PDP,Yes,No,Yes,OMES EMPLOYEES GROUP INSURANCE DIVISION,HealthChoice,OMES Employees Group Insurance Division (Employer PDP),State of Oklahoma,01/01/2007 0:00:00,NA,NA,NA,NA,NA,NA,1,2015


5. Initialize List for Yearly Data 

In [86]:
# Initialize an empty list to store yearly data
yearly_data_list <- list()

6. Load and Collapse Data for Each Year 

In [87]:
# Load and collapse data for each year
for (year in years) {
  # Load monthly data for the current year
  data <- map_dfr(monthlist, ~load_month(.x, year))
  
  # Filter out unwanted plans
  filtered_data <- data %>%
    filter(!`Plan Type` %in% c("SNP", "800-series", "Prescription Drug Only"))

  # Summarize Count distinct plans per county for the current year
  yearly_summary <- filtered_data %>%
    group_by(`State`, `County`) %>%
    summarize(plan_count = n_distinct(`Plan ID`), .groups = "drop") %>%
    mutate(year = year)  # Add year to the summary

  # Append the current year's summary to the list
  yearly_data_list[[as.character(year)]] <- yearly_summary
}

[1] "CPSC_Contract_Info_2014_01.csv"
[1] "CPSC_Enrollment_Info_2014_01.csv"
[1] "CPSC_Contract_Info_2014_02.csv"
[1] "CPSC_Enrollment_Info_2014_02.csv"
[1] "CPSC_Contract_Info_2014_03.csv"
[1] "CPSC_Enrollment_Info_2014_03.csv"
[1] "CPSC_Contract_Info_2014_04.csv"
[1] "CPSC_Enrollment_Info_2014_04.csv"
[1] "CPSC_Contract_Info_2014_05.csv"
[1] "CPSC_Enrollment_Info_2014_05.csv"
[1] "CPSC_Contract_Info_2014_06.csv"
[1] "CPSC_Enrollment_Info_2014_06.csv"
[1] "CPSC_Contract_Info_2014_07.csv"
[1] "CPSC_Enrollment_Info_2014_07.csv"
[1] "CPSC_Contract_Info_2014_08.csv"
[1] "CPSC_Enrollment_Info_2014_08.csv"
[1] "CPSC_Contract_Info_2014_09.csv"
[1] "CPSC_Enrollment_Info_2014_09.csv"
[1] "CPSC_Contract_Info_2014_10.csv"
[1] "CPSC_Enrollment_Info_2014_10.csv"
[1] "CPSC_Contract_Info_2014_11.csv"
[1] "CPSC_Enrollment_Info_2014_11.csv"
[1] "CPSC_Contract_Info_2014_12.csv"
[1] "CPSC_Enrollment_Info_2014_12.csv"
[1] "CPSC_Contract_Info_2015_01.csv"
[1] "CPSC_Enrollment_Info_2015_01.csv"
[1] "CPSC_Co

7. Combine All Yearly Summaries into a Single Dataframe

In [88]:
# Combine all yearly summaries into a single dataframe
final_summary <- bind_rows(yearly_data_list)

8. Create a Boxplot of Plan Counts by County Over the Years

In [89]:
# Create a boxplot of plan counts by county over the years
enrollment_plot <- ggplot(final_summary, aes(x = factor(year), y = plan_count)) +
  geom_boxplot() +
  labs(title = "Distribution of Plan Counts by County (2014-2019)",
       x = "Year", y = "Number of Plans")


9. Specify the Output Directory for Visualizations

In [90]:
# Specify the output directory for visualizations
output_analysis_dir <- "/home/salizad/econ470/a0/work/Homework2/submission1/analysis"

# Create the directory structure if it doesn't exist
dir.create(output_analysis_dir, recursive = TRUE, showWarnings = FALSE)

10. Output Results to the Console and Save Files 

In [91]:
# Output results to the console
if (nrow(final_summary) == 0) {
  cat("No data loaded.\n")
} else {
  cat("Data loaded successfully:\n")
  print(head(final_summary))  # Show the first few rows
  
  # Save the summary to a CSV file
  write.csv(final_summary, file.path(output_analysis_dir, "enrollment_data_summary.csv"), row.names = FALSE)
  cat("Summary saved to 'enrollment_data_summary.csv'.\n")
  
  # Save the plot to the specified analysis directory
  ggsave(filename = file.path(output_analysis_dir, "enrollment_boxplot.png"), plot = enrollment_plot)
  cat("Boxplot saved to 'enrollment_boxplot.png'.\n")
}


Data loaded successfully:
# A tibble: 6 × 4
  State County plan_count  year
  <chr> <chr>       <int> <int>
1 NA    NA            373  2014
2 NA    NA            360  2015
3 NA    NA            354  2016
4 NA    NA            353  2017
5 NA    NA            400  2018
6 NA    NA            428  2019
Summary saved to 'enrollment_data_summary.csv'.


Saving 6.67 x 6.67 in image


Boxplot saved to 'enrollment_boxplot.png'.


## 2. Analyzing Plan Bids in 2014 and 2018

1. Loading LAndscape Data

1.2 Loading necessary libraries

In [93]:
library(readr)
library(readxl)
library(stringr)


1.1 Loading MA Landscape Data for 2014

In [94]:
setwd("/home/salizad/econ470/a0/work/ma-data/ma/landscape/Extracted Data/")

In [95]:
ma.path.2014 <- "/home/salizad/econ470/a0/work/ma-data/ma/landscape/Extracted Data/2014LandscapeSource file MA_AtoM 05292014.csv"
ma.data.2014 <- read_csv(ma.path.2014,
                          skip=6,
                          col_names=c("state","county","org_name","plan_name","plan_type","premium","partd_deductible",
                                      "drug_type","gap_coverage","drug_type_detail","contractid",
                                      "planid","segmentid","moop","star_rating"),
                          col_types = cols(
                            state = col_character(),
                            county = col_character(),
                            org_name = col_character(),
                            plan_name = col_character(),
                            plan_type = col_character(),
                            premium = col_character(),
                            partd_deductible = col_character(),
                            drug_type = col_character(),
                            gap_coverage = col_character(),
                            drug_type_detail = col_character(),
                            contractid = col_character(),
                            planid = col_double(),
                            segmentid = col_double(),
                            moop = col_character(),
                            star_rating = col_character()
                          )) %>%
  mutate_at(c('premium','partd_deductible'), ~str_replace(.,"-","0")) %>%
  mutate_at(c('premium','partd_deductible'), ~parse_number(.))

1.2 Loading MA Landscape Data for 2018

In [102]:
ma.path.2018.a <- "/home/salizad/econ470/a0/work/ma-data/ma/landscape/Extracted Data/2018LandscapeSource file MA_AtoM 10142017.csv"

ma.data.2018.a <- read_csv(ma.path.2018.a,
                            skip = 6,
                            col_names = c("state", "county", "org_name", "plan_name", "plan_type", 
                                           "premium", "partd_deductible", "drug_type", "gap_coverage", 
                                           "drug_type_detail", "contractid", "planid", "segmentid", 
                                           "moop", "star_rating"),
                            col_types = cols(
                              state = col_character(),
                              county = col_character(),
                              org_name = col_character(),
                              plan_name = col_character(),
                              plan_type = col_character(),
                              premium = col_character(),
                              partd_deductible = col_character(),
                              drug_type = col_character(),
                              gap_coverage = col_character(),
                              drug_type_detail = col_character(),
                              contractid = col_character(),
                              planid = col_double(),
                              segmentid = col_double(),
                              moop = col_character(),
                              star_rating = col_character()
                            )) %>%
  mutate(across(c('premium', 'partd_deductible'), ~str_replace(., "-", "0"))) %>%
  mutate(across(c('premium', 'partd_deductible'), ~parse_number(.)))

2. Loading Risk/Rebate Data

2.1 Load Part C and Part D Risk/Rebate Data for 2014

In [108]:
# Load Part C Risk/Rebate Data for 2014
risk.rebate.path.2014 <- "/home/salizad/econ470/a0/work/ma-data/ma/cms-payment/2014/2014PartCPlan Level.xlsx"
risk.rebate.2014 <- read_xlsx(risk.rebate.path.2014, range="A4:G3620", sheet="result.srx",  # Use the correct sheet name
                               col_names=c("contractid","planid","contract_name","plan_type",
                                           "riskscore_partc","payment_partc","rebate_partc"))

# Load Part D Risk/Rebate Data for 2014
risk.rebate.path.2014D <- "/home/salizad/econ470/a0/work/ma-data/ma/cms-payment/2014/2014PartDPlans.xlsx"
risk.rebate.2014D <- read_xlsx(risk.rebate.path.2014D, range="A4:H4460", sheet="Sheet2",  # Use the correct sheet name
                               col_names=c("contractid","planid","contract_name","plan_type",
                                           "directsubsidy_partd","riskscore_partd","reinsurance_partd",
                                           "costsharing_partd"))

In [112]:
# Inspect the data frame
print(head(risk.rebate.2014D))  # View the first few rows
print(colnames(risk.rebate.2014D))  # Print column names

# A tibble: 0 × 0
character(0)


2.2 Load Part C and Part D Risk/Rebate Data for 2018

In [109]:
# Load Part C Risk/Rebate Data for 2018
risk.rebate.path.2018 <- "/home/salizad/econ470/a0/work/ma-data/ma/cms-payment/2018/2018PartCPlanLevel.xlsx"
risk.rebate.2018 <- read_xlsx(risk.rebate.path.2018, range="A4:G3620", sheet="result.srx",
                               col_names=c("contractid","planid","contract_name","plan_type",
                                           "riskscore_partc","payment_partc","rebate_partc"))

# Load Part D Risk/Rebate Data for 2018
risk.rebate.path.2018D <- "/home/salizad/econ470/a0/work/ma-data/ma/cms-payment/2018/2018PartDPlans.xlsx"
risk.rebate.2018D <- read_xlsx(risk.rebate.path.2018D, range="A4:H4460", sheet="result.srx",
                               col_names=c("contractid","planid","contract_name","plan_type",
                                           "directsubsidy_partd","riskscore_partd","reinsurance_partd",
                                           "costsharing_partd"))

3. Cleaning and Preparing Risk/Rebate Data

3.1 Cleaning Part C and D Data for 2014

In [111]:
# Clean Part C Data for 2014
risk.rebate.2014 <- risk.rebate.2014 %>%
  mutate(
    across(c(riskscore_partc, payment_partc, rebate_partc),
           ~ parse_number(as.character(.))),
    planid = as.numeric(planid),
    year   = 2014
  ) %>%
  select(contractid, planid, contract_name, plan_type,
         riskscore_partc, payment_partc, rebate_partc, year)

# Clean Part D Data for 2014
risk.rebate.2014D <- risk.rebate.2014D %>%
  mutate(
    across(c(directsubsidy_partd, reinsurance_partd, costsharing_partd),
           ~ parse_number(as.character(.))),
    payment_partd = directsubsidy_partd + reinsurance_partd + costsharing_partd,
    planid        = as.numeric(planid)
  ) %>%
  select(contractid, planid, payment_partd,
         directsubsidy_partd, reinsurance_partd, costsharing_partd,
         riskscore_partd)

ERROR: [1m[33mError[39m in `mutate()`:[22m
[1m[22m[36mℹ[39m In argument: `across(...)`.
[1mCaused by error in `across()`:[22m
[33m![39m Can't subset columns that don't exist.
[31m✖[39m Column `directsubsidy_partd` doesn't exist.
